# 🛠️ DBeaver Launcher s kontrolou procesu

Tento notebook spustí DBeaver a ověří, zda proces skutečně běží.

In [ ]:
import subprocess
import os
import time
import csv
from datetime import datetime
import uuid
import psutil
import pyautogui

def log_result(msg, st):
    log_file = "automation_log.csv"
    now = datetime.now()
    rid = str(uuid.uuid4())[:8]
    
    # Capturing screenshot
    ss_dir = "screenshots"
    if not os.path.exists(ss_dir): os.makedirs(ss_dir)
    ss_name = f"screenshot_{rid}.png"
    ss_path = os.path.join(ss_dir, ss_name)
    try:
        pyautogui.screenshot(ss_path)
        print(f"📸 Screenshot saved: {ss_path}")
    except Exception as e:
        print(f"❌ Error taking screenshot: {e}")
        ss_path = ""

    header = ["id_behu", "datum", "cas", "error_message", "status", "screenshot"]
    # Relative path for the server
    relative_ss_path = f"/screenshots/{ss_name}" if ss_path else ""
    
    new_entry = [rid, now.strftime("%d.%m.%Y"), now.strftime("%H:%M:%S"), msg, st, relative_ss_path]
    
    file_exists = os.path.isfile(log_file)
    with open(log_file, mode='a', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        if not file_exists: writer.writerow(header)
        writer.writerow(new_entry)
    print(f"📝 Log updated: {st} ({msg})")

def is_dbeaver_running():
    for proc in psutil.process_iter(['name']):
        if proc.info['name'] and 'dbeaver' in proc.info['name'].lower():
            return True
    return False

def launch_dbeaver_with_check():
    path = r"C:\Program Files\DBeaver\dbeaver.exe"
    
    if not os.path.exists(path):
        log_result("DBeaver path does not exist", "error")
        return

    for proc in psutil.process_iter(['name']):
        if proc.info['name'] and 'dbeaver' in proc.info['name'].lower():
            proc.kill()
    
    time.sleep(1)
    
    print(f"🚀 Launching DBeaver...")
    subprocess.Popen(["explorer.exe", path])
    
    for i in range(10):
        time.sleep(1)
        if is_dbeaver_running():
            print(f"✅ DBeaver detected at {i+1}s.")
            log_result("DBeaver launched and confirmed", "ok")
            return
    
    print("❌ DBeaver failed to start within 10s.")
    log_result("DBeaver did not start in time", "error")

launch_dbeaver_with_check()